# Test Case LVV-t1992
This case will verify that the M2/Camera Hexapod is minimally functional.
The blocks below represent different steps of the test case.

Requirements
* EFD
* Hexapod(s) powered on
* Thermal sensors attached to the six actuators of the hexapod.
* CSC running

This test will require manual verification of certain events and telemetry in the summit EFD.
Also manual verification of appropriate temperatures for each actuator.

In [ ]:
from lsst.ts import salobj
from lsst.ts.idl.enums import MTHexapod
import logging
import asyncio
import os
import yaml

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd
import astropy.units as u

The following block sets the necessary environment variables for setting up the DDS/SAL communication

In [ ]:
os.environ["LSST_DDS_HISTORYSYNC"] = "30"

In [ ]:
STD_WAIT = 39 # This is the amount of time to wait in between a move just to make sure that the actuators are not overheating

In [ ]:
log = logging.getLogger(__name__) # This sets up the logger for the test.

In [ ]:
test_message = "Camera Hexapod Integration Test"
script = salobj.Controller("Script", index=42658885)

This is how you start the remote for the CSC.

In [ ]:
domain = salobj.Domain()
print (domain)
#index=1 is the camera hexapod!! index=2 is the M2 hexapod!
hexapod_csc = salobj.Remote(name="MTHexapod", domain=domain, index=1)
print (hexapod_csc)
await hexapod_csc.start_task
#This is to bring the state machine in the right starting state
await salobj.set_summary_state(hexapod_csc, salobj.State.ENABLED)
#await salobj.set_summary_state(hexapod_csc, salobj.State.OFFLINE)

In [ ]:
def enabled_substate_callback(evt):
    """Print the enabled substate when event is received."""
    print(MTHexapod.EnabledSubstate(evt.enabledSubstate))

In [ ]:
def in_position_callback(evt):
    """Print the in position event when it is received."""
    print(evt.in_position)

This next block will make sure that the CSC is ready for the test

In [ ]:
connected = await hexapod_csc.evt_connected.aget() # This is how you get an event/telemetry with await remote.type_name.aget() type being [evt,tel] and name being the name of the topic
commandable = connected.command
telemetry_working = connected.telemetry

if not connected or not telemetry_working:
    raise Exception("Hexapod not connected or telemetry not being received.")

    #This is only for the real system. The simulater only has the AVAILABLE offline state 
'''   
controller_state = await hexapod_csc.evt_controllerState.aget()
state = controller_state.controllerState
offline_substate = controller_state.offlineSubstate
print(state)
if not state == salobj.State.OFFLINE and not offline_substate == MTHexapod.OfflineSubstate.AVAILABLE:
    raise Exception("Controller must be changed to  Offline/Available Substate.")
'''
commandable_by_dds = await hexapod_csc.evt_commandableByDDS.aget()
dds_state = commandable_by_dds.state
if not dds_state:
    raise Exception("Controller must in CommandableByDDS state.")


The next four blocks bring the CSC to the Enabled state.

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.OFFLINE)

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.DISABLED)

In [ ]:
await hexapod_csc.cmd_enterControl.set_start() # Send the enterControl command notice the casing and the set_start method

In [ ]:
await hexapod_csc.cmd_start.set_start() 

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.ENABLED)

In [ ]:
await hexapod_csc.cmd_clearError.set_start() # This clears the error

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
# Step skiped see deviation LVV-T1802 Test Step 5                                  
hexapod_csc.evt_controllerState.callback = enabled_substate_callback                                
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=200, u=0,v=0,w=0,sync=True)

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1992 Test Step 7 - Starting time: {now} UTC")
hexapod_csc.evt_controllerState.callback = enabled_substate_callback                                

await hexapod_csc.cmd_move.set_start(x=500,y=-500,z=200, u=0.01,v=-0.015,w=0,sync=True)
script.log.info(f"STOP- {test_message} -- LVV-T1992 Test Step 7 - Ending time: {now} UTC")

In [ ]:
await asyncio.sleep(STD_WAIT)
# thermal sensors

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
# thermal sensors

In [ ]:
#This command is to set the Hexapod to zero position
now = datetime.now()
script.log.info(f"START -- {test_message} -- LVV-T1992 -- Move to X,Y,Z,U,V,W=0 -- Starting Time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

This block moves the hexapod to a large position but stops it after waiting 3 seconds

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1992 -- Stop command test -- Starting Time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=5000,u=0,v=0,w=0,sync=True)
await asyncio.sleep(3)
await hexapod_csc.cmd_stop.set_start()
script.log.info(f"STOP- {test_message} -- LVV-T1992 -- Stop command test -- Finishing Time: {now} UTC")

In [ ]:
#Step 15
await asyncio.sleep(STD_WAIT)

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
#This step requires telemetry from the mount and the rotator to work see.

script.log.info(f"START- {test_message} -- LVV-T1802 Test setCompensationMode")

To moves the hexapod via the lookup table use /hdrass/Camera_Hexapod/hex_diagnostics.ipynb

In [ ]:
script.log.info(f"STOP- {test_message} -- LVV-T1802 Test setCompensationMode")

In [ ]:
#Step 22
await asyncio.sleep(STD_WAIT)
# thermal sensors

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [ ]:
#This command is to set the Hexapod to zero position
now = datetime.now()
script.log.info(f"START -- {test_message} -- LVV-T1992 -- Move to X,Y,Z,U,V,W=0 -- Starting Time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

In [ ]:
now = datetime.now()
script.log.info(f"START- {test_message} -- LVV-T1992 -- offset command test -- Starting time: {now} UTC")
await hexapod_csc.cmd_move.set_start(x=500,y=800,z=200,u=0,v=0,w=0,sync=True)


In [ ]:
await hexapod_csc.cmd_offset.set_start(x=0,y=0,z=500,u=0,v=0,w=0,sync=True)
script.log.info(f"STOP- {test_message} -- LVV-T1992 -- offset command test -- Finishing time: {now} UTC")

In [ ]:
#Step 27
await asyncio.sleep(STD_WAIT)

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.STANDBY) # Transition the CSC to Standby state
await domain.close() # Close the remote connection

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |